In [ ]:
import tensorflow as tf
import keras_cv
from tensorflow import keras

In [ ]:
# load the pipeline, get text_encoder and decoder

model = keras_cv.models.StableDiffusionV2(img_width=768, img_height=768)
text_encoder_model = model.text_encoder
decoder_model = model.decoder

In [ ]:
# convert the two models to tflite

converter1 = tf.lite.TFLiteConverter.from_keras_model(text_encoder_model)
tflite_text_encoder = converter1.convert()

converter2 = tf.lite.TFLiteConverter.from_keras_model(decoder_model)
tflite_decoder = converter2.convert()

with open('/tmp/sd2_text_encoder.tflite', 'wb') as f:
    f.write(tflite_text_encoder)
    
with open('/tmp/sd2_decoder.tflite', 'wb') as f:
    f.write(tflite_decoder)

In [ ]:
text_encoder_model.save('/tmp/sd2/text_encoder')
decoder_model.save('/tmp/sd2/decoder')

In [ ]:
text_encoder_model = tf.saved_model.load('/tmp/sd2/text_encoder/')
concrete_func = text_encoder_model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
concrete_func.inputs[0].set_shape([1, 77])
concrete_func.inputs[1].set_shape([1, 77])
converter1 = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_text_encoder = converter1.convert()

decoder_model = tf.saved_model.load('/tmp/sd2/decoder/')
concrete_func = decoder_model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
concrete_func.inputs[0].set_shape([1, 96, 96, 4])
converter2 = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_decoder = converter2.convert()

with open('/tmp/sd2_text_encoder_fixed_batch.tflite', 'wb') as f:
    f.write(tflite_text_encoder)
    
with open('/tmp/sd2_decoder_fixed_batch.tflite', 'wb') as f:
    f.write(tflite_decoder)